In [10]:
import json

# File paths for the provided JSON files
file1_path = 'qlora_finetuning_dataset_19_11.json'
file2_path = r'D:\Master_Thesis\final_pipeline\new_data\new_train_output.json'  # Use raw string for Windows paths

# Read and parse the JSON file to extract UserID from the first file
with open(file1_path, 'r') as file1:
    json_data1 = json.load(file1)
    user_ids_file_1 = []
    for entry in json_data1:
        for user in entry.get("users", []):
            if "UserID" in user:
                user_ids_file_1.append(int(user["UserID"]))  # Ensure the value is treated as an integer index

# Read and parse the JSON file to extract all user IDs from the second file
with open(file2_path, 'r') as file2:
    json_data2 = json.load(file2)
    user_ids_file_2 = [entry["user_id"] for entry in json_data2 if "user_id" in entry]

# Match User IDs using the first JSON as index for the second JSON
matched_ids = {}
for user_id_1 in user_ids_file_1:
    if 0 <= user_id_1 < len(user_ids_file_2):  # Ensure index is within bounds
        matched_ids[user_id_1] = user_ids_file_2[user_id_1-1]

# Print the user ids from the first file
print("User IDs from first JSON file:")
print(user_ids_file_1)

# Print the user ids array from the second file
print("\nUser IDs from second JSON file:")
print(user_ids_file_2)

# Print matched user IDs
print("\nMatched User IDs:")
for key, value in matched_ids.items():
    print(f"{key}: {value}")


User IDs from first JSON file:
[6, 70, 218, 176, 56, 60, 63, 75, 103, 193, 196, 200, 22, 25, 54, 57, 167, 85, 90, 127, 53, 73, 115, 165, 210, 212, 89, 174, 192, 235, 11, 172, 237, 110, 98, 113, 49, 126, 162, 247, 2, 35, 93, 183, 189, 205, 59, 117, 198, 223]

User IDs from second JSON file:
['AFSKPY37N3C43SOI5IEXEK5JSIYA', 'AHV6QCNBJNSGLATP56JAWJ3C4G2A', 'AFJBKPK5W56XWSNPQU2WW66ISWYQ', 'AFXF3EGQTQDXMRLDWFU7UBFQZB7Q', 'AFWVN52MRBWOTIK7UGXBWGOY4HBA', 'AFQQQ5LGNSQUEBGDCYBAZZE5T3DA', 'AGAM2CCKV52HI4YZU7ASZTSXA7YQ', 'AF2BLE54TEMGZ546U763ZHZRXC4A', 'AGZZXSMMS4WRHHJRBUJZI4FZDHKQ', 'AGD25H7BIT2JUXSIOPYCYB23J3ZQ', 'AEXGISIVX7WBUNI7UHHERVB3DF7Q', 'AEZP6Z2C5AVQDZAJECQYZWQRNG3Q', 'AGTW6ZGPUAORQ7X6CNBP6PJW7OTA', 'AHALZ7AKVAVL7QEVBCI55JVLGXOQ', 'AHTLWVDXSMG5YMVMEIWWOU6XBZMA', 'AHMG3ALUBE3FEBHODTBHP5J24YDA', 'AG73BVBKUOH22USSFJA5ZWL7AKXA', 'AH7LDXK3GT67FZKMRW7MP477DJVQ', 'AE3KLVXGZPANXE5XLXYKHTVAZ3FQ', 'AETWQ4GYGV4QRMTMWGBIDVASBKHA', 'AH6JLX5SJSUOJBBT6RRE2RDXIG6A', 'AF2YKZQRMRGJ655I3MKQUYFGRQGA', 'AHW

In [18]:
import json

# File paths for the provided JSON files
file1_path = 'filtered_data_excluding_description.json'
file2_path = r'D:\Master_Thesis\final_pipeline\new_data\new_train_output.json'
output_file_path = 'new_candidate_items_with_profile.json'  # Path for the new JSON file

# Read and parse the JSON file to extract UserID from the first file
with open(file1_path, 'r') as file1:
    json_data1 = json.load(file1)

# Read and parse the JSON file to extract all user IDs from the second file
with open(file2_path, 'r') as file2:
    json_data2 = json.load(file2)
    user_ids_file_2 = [entry["user_id"] for entry in json_data2 if "user_id" in entry]

# Create the new JSON structure
new_json_data = []

for entry in json_data1:
    for user in entry.get("users", []):
        user_id_index = int(user["UserID"])  # Get the UserID as an integer
        if 1 <= user_id_index <= len(user_ids_file_2):  # Check index bounds (adjusted for 1-based index)
            user_asin = user_ids_file_2[user_id_index - 1]  # Adjust for 0-based indexing
            extracted_products = eval(user.get("Extracted Products", "[]"))  # Convert string to list
            candidate_items = {str(i + 1): product for i, product in enumerate(extracted_products)}
            
            # Construct User_Profile
            short_term = user.get("Profile Sections", {}).get("Short-Term Interests", "")
            long_term = user.get("Profile Sections", {}).get("Long-term Preferences", "")
            profile_summary = user.get("Profile Sections", {}).get("User Profile Summary", "")
            user_profile = (
                f"\"Short-Term Interests\": {short_term}\n"
                f"\"Long-Term Preferences\": {long_term}\n"
                f"\"User_Profile\": {profile_summary}"
            )
            
            # Construct the new JSON entry
            new_entry = {
                "User_ID": user_asin,
                "User_Profile": user_profile,
                "Candidate_Items": candidate_items
            }
            
            new_json_data.append(new_entry)

# Write the new JSON data to a file
with open(output_file_path, 'w') as output_file:
    json.dump(new_json_data, output_file, indent=4)

print(f"New JSON file with user profiles saved to: {output_file_path}")

New JSON file with user profiles saved to: new_candidate_items_with_profile.json


In [2]:
import json

def print_sorted_user_ids(json_file_path):
    """
    Reads a JSON file and prints all the UserIDs in sorted order.
    
    Args:
        json_file_path (str): Path to the JSON file.
    """
    try:
        # Load JSON data
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        
        # Collect all UserIDs
        user_ids = []
        for entry in data:
            for user in entry.get("users", []):
                user_ids.append(int(user["UserID"]))  # Convert to int for sorting

        # Sort and print UserIDs
        user_ids.sort()
        print("Sorted UserIDs:", user_ids)
    except Exception as e:
        print("An error occurred:", e)

# Example usage
json_file_path = "qlora_finetuning_dataset_19_11_with_id.json"
print_sorted_user_ids(json_file_path)


Sorted UserIDs: [2, 6, 11, 22, 25, 35, 49, 53, 54, 56, 57, 59, 60, 63, 70, 73, 75, 85, 89, 90, 93, 98, 103, 110, 113, 115, 117, 126, 127, 162, 165, 167, 172, 174, 176, 183, 189, 192, 193, 196, 198, 200, 205, 210, 212, 218, 223, 235, 237, 247]


In [5]:
import json

def check_duplicate_user_ids_from_file(file_path):
    """
    Reads a JSON file and checks for duplicate user_ids.
    
    Parameters:
        file_path (str): Path to the JSON file.
        
    Returns:
        list: A list of duplicate user_ids, or an empty list if no duplicates are found.
    """
    try:
        with open(file_path, 'r') as file:
            json_data = json.load(file)
        
        # Extract user_ids and check for duplicates
        user_ids = [entry['User_ID'] for entry in json_data]
        duplicates = [user_id for user_id in set(user_ids) if user_ids.count(user_id) > 1]
        return duplicates

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except json.JSONDecodeError:
        print(f"Invalid JSON format in file: {file_path}")
        return []

# Example usage
file_path = 'new_candidate_items_with_profile.json'  # Replace with the path to your JSON file
duplicates = check_duplicate_user_ids_from_file(file_path)
if duplicates:
    print("Duplicate user_ids found:", duplicates)
else:
    print("No duplicate user_ids found.")


No duplicate user_ids found.


In [11]:
import json
import os

def analyze_json_files_with_mapping(
    main_source_path: str,
    runs_paths: list,
    user_mapping_path: str,
    output_path: str = "updated_matching_ids_pipeline.json"
):
    """
    Analyzes the main source JSON (matching_ids_pipeline.json) and a list of
    'results' JSON files (_run_1, _run_2, etc.) using a user mapping file
    (user_mapping_beauty.json). For each user with Recall@10 > 0:

      1. Convert the numeric user_id (from the results JSON) to the real user ID 
         using user_mapping_beauty.json.
      2. Check whether the real user ID already exists in the main source.
      3. If not, add that new user entry (profile + candidate items) to the main source 
         in the same schema.

    Finally:
      - Writes the updated main source to a new JSON file.
      - Reads that new file back and checks the structure to ensure correctness.
    """

    # -------------------
    # 1. Load the main source
    # -------------------
    with open(main_source_path, 'r', encoding='utf-8') as f:
        main_source = json.load(f)

    original_length = len(main_source)
    print(f"[DEBUG] Original main source length: {original_length}")

    # Convert main_source to a dict {User_ID -> user_object} for quick membership checking
    main_source_dict = {user["User_ID"]: user for user in main_source}

    # -------------------
    # 2. Load and invert user_mapping_beauty.json
    # -------------------
    with open(user_mapping_path, 'r', encoding='utf-8') as f:
        mapping_dict = json.load(f)
    # e.g., { "AFQQQ5LGNSQUEBGDCYBAZZE5T3DA": 6, ... }
    # We want to invert to get { 6: "AFQQQ5LGNSQUEBGDCYBAZZE5T3DA", ... }
    inverted_mapping = {v: k for k, v in mapping_dict.items()}

    # Keep track of newly added users for debug output
    newly_added_user_ids = []

    # -------------------
    # 3. Process each results file (_run_X)
    # -------------------
    for run_path in runs_paths:
        if not os.path.exists(run_path):
            print(f"[DEBUG] Warning: File not found: {run_path}")
            continue

        with open(run_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        results = data.get("results", [])

        # 4. Check each user in 'results'
        for user_data in results:
            evaluation = user_data.get("evaluation", {})
            recalls = evaluation.get("recalls", {})
            recall_at_10 = recalls.get("Recall@10", 0.0)

            # If recall_at_10 > 0, that indicates a match
            if recall_at_10 > 0:
                numeric_user_id = user_data.get("user_id")
                try:
                    numeric_user_id = int(numeric_user_id)
                except (TypeError, ValueError):
                    print(f"[DEBUG] user_id '{numeric_user_id}' is invalid. Skipping.")
                    continue

                # Look up real_user_id from the inverted mapping
                real_user_id = inverted_mapping.get(numeric_user_id)
                if not real_user_id:
                    print(f"[DEBUG] numeric user_id={numeric_user_id} not in mapping. Skipping.")
                    continue

                # If the user_id is already in main_source_dict, skip
                if real_user_id in main_source_dict:
                    print(f"[DEBUG] Duplicate user found. Skipping User_ID: {real_user_id}")
                else:
                    # Create a new user entry
                    new_user = {
                        "User_ID": real_user_id,
                        "User_Profile": user_data.get("profile", ""),
                        "Candidate_Items": {}
                    }
                    # Convert extracted_products to Candidate_Items
                    extracted_products = user_data.get("extracted_products", [])
                    for i, product in enumerate(extracted_products, start=1):
                        new_user["Candidate_Items"][str(i)] = product

                    main_source_dict[real_user_id] = new_user
                    newly_added_user_ids.append(real_user_id)

    # -------------------
    # 5. Re-convert to list, write to JSON
    # -------------------
    updated_main_source = list(main_source_dict.values())

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(updated_main_source, f, indent=2, ensure_ascii=False)

    # Debug info
    print(f"[DEBUG] Newly added user IDs: {newly_added_user_ids}")
    print(f"[DEBUG] Number of newly added users: {len(newly_added_user_ids)}")
    print(f"[DEBUG] Updated main source length: {len(updated_main_source)}")
    print(f"[DEBUG] Updated main source written to: {output_path}")

    # -------------------
    # 6. Read the new file and check structure
    # -------------------
    if os.path.exists(output_path):
        with open(output_path, "r", encoding="utf-8") as f:
            final_data = json.load(f)

        # Validate each entry’s structure
        # Ensure each item is a dict with keys: User_ID, User_Profile, Candidate_Items
        correct_structure = True
        for idx, entry in enumerate(final_data):
            if not isinstance(entry, dict):
                print(f"[ERROR] Entry at index {idx} is not a dict.")
                correct_structure = False
                break

            required_keys = {"User_ID", "User_Profile", "Candidate_Items"}
            if not required_keys.issubset(entry.keys()):
                print(f"[ERROR] Entry at index {idx} does not have the required keys {required_keys}.")
                correct_structure = False
                break

            if not isinstance(entry["Candidate_Items"], dict):
                print(f"[ERROR] 'Candidate_Items' at index {idx} is not a dict.")
                correct_structure = False
                break

        if correct_structure:
            print("[DEBUG] Structure is correct for all entries in the updated file.")
        else:
            print("[DEBUG] Structure problems were found. Please check the error messages above.")
    else:
        print(f"[DEBUG] Could not verify structure because {output_path} was not found.")

    # Return the updated list in memory, in case you need it
    return updated_main_source


# -----------------------
# Example Usage
# -----------------------
if __name__ == "__main__":
    # Example paths
    main_path = r"matching_ids_pipeline.json"
    user_mapping_path = r"user_mapping_beauty.json"

    runs = [
        r"D:\Master_Thesis\final_pipeline\results_full_review\results_beauty_without_description_no_adapter_reviews_10_sample_None_2024-12-28_run_1_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_full_review\results_beauty_without_description_no_adapter_reviews_10_sample_None_2024-12-28_run_2_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_full_review\results_beauty_without_description_no_adapter_reviews_10_sample_None_2024-12-28_run_3_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_full_review\results_beauty_without_description_no_adapter_reviews_10_sample_None_2024-12-28_run_4_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_full_review\results_beauty_without_description_no_adapter_reviews_10_sample_None_2024-12-28_run_5_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_beauty_with_description_no_adapter_reviews_10_sample_None_2025-01-07_run_1_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_beauty_with_description_no_adapter_reviews_10_sample_None_2025-01-07_run_2_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_beauty_with_description_no_adapter_reviews_10_sample_None_2025-01-07_run_3_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_beauty_without_description_no_adapter_reviews_100_sample_None_2025-01-08_run_1_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_beauty_without_description_no_adapter_reviews_100_sample_None_2025-01-08_run_2_with_full_reviews.json",
        r"D:\Master_Thesis\final_pipeline\results_beauty_without_description_no_adapter_reviews_100_sample_None_2025-01-08_run_3_with_full_reviews.json"
    ]
    output_file = r"D:\Master_Thesis\final_pipeline\QLoRa_finetuning\updated_matching_ids_pipeline.json"

    updated_source = analyze_json_files_with_mapping(
        main_source_path=main_path,
        runs_paths=runs,
        user_mapping_path=user_mapping_path,
        output_path=output_file
    )


[DEBUG] Original main source length: 46
[DEBUG] Duplicate user found. Skipping User_ID: AEXGISIVX7WBUNI7UHHERVB3DF7Q
[DEBUG] Duplicate user found. Skipping User_ID: AEUAQ6BAUEBP2C754LR3ILYHP54Q
[DEBUG] Duplicate user found. Skipping User_ID: AFGGRGHGHSFKMZASFBYFRHBBEOPA
[DEBUG] Duplicate user found. Skipping User_ID: AEKNMPUXBMNMHDYWMLKOUTEY3LYA
[DEBUG] Duplicate user found. Skipping User_ID: AFFGJTTHRCSKGMMCNHT7UWTMKKCQ
[DEBUG] Duplicate user found. Skipping User_ID: AHSV4TYSAX52BIHH7PLZRD44KZHA
[DEBUG] Duplicate user found. Skipping User_ID: AEXGISIVX7WBUNI7UHHERVB3DF7Q
[DEBUG] Duplicate user found. Skipping User_ID: AETWQ4GYGV4QRMTMWGBIDVASBKHA
[DEBUG] Duplicate user found. Skipping User_ID: AHWCZ47A7FIYEJ7KEJG3BO3F5YXQ
[DEBUG] Duplicate user found. Skipping User_ID: AGFXY5HVXFJFJ2SJDGZL5YLGR6RA
[DEBUG] Duplicate user found. Skipping User_ID: AGCV6N4LVTEQVYSE3LXR3J2QB3EA
[DEBUG] Duplicate user found. Skipping User_ID: AF5A5PMCP3EPIOVGZSAJWNICC6UQ
[DEBUG] Duplicate user found. Skippi